In [ ]:
import json
import os
import requests
from pydantic import BaseModel, Field
from typing import Optional
from dotenv import load_dotenv
from rich import print
from litellm import completion
from arch_prompts import PROMPTS


load_dotenv('.env')
pplx_api_key = os.environ['PERPLEXITY_API_KEY']
gemini_api_key = os.environ['GEMINI_API_KEY']

In [ ]:
class Agent:
    def __init__(self, sys_prompt: str, model: str = "gpt-4o"):
        self.sys_prompt = sys_prompt
        self.messages = [{"role": "system", "content": sys_prompt}]
        self.model = model

    def llm_call(self, prompt: str, response_format = None) -> str:
        self.messages.append({"role": "user", "content": prompt})
        try:
            response = completion(
                model=self.model,
                messages=self.messages,
                temperature=0.1,
                response_format=response_format
            )
            response_content = response.choices[0].message.content
            self.messages.append({"role": "assistant", "content": response_content})
            return response_content
        except Exception as e:
            print(f"LLM call failed: {e}")
            return ""

#### Input

In [ ]:
firm_name = "Ware Malcomb"
resume = ""
portfolio = ""

#### Fetch PPLX Email and Description

In [ ]:
url = "https://api.perplexity.ai/chat/completions"
headers = {"Authorization": f"Bearer {pplx_api_key}"}

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
    

class ArchFirm(BaseModel):
    recruiting_email: Optional[str] = Field(None, description="What is the recruiting or careers contact email address listed on the firm's website or careers page?")
    firm_description: Optional[str] = Field(None, description="What is the detailed description of the architecture firm, including their focus areas, specializations, notable projects, design philosophy, and overall approach to architecture?")

payload = {
    "model": "sonar-reasoning-pro",
    "messages": [
        {"role": "system", "content": "For a given architecture firm, your task is to extract a complete description about the firm including their focus areas, specializations, notable projects, design philosophy, and overall approach to architecture? Be precise and concise."},
        {"role": "user", "content": (
            f"Please extract info for : {firm_name}"

        )},
    ],
}


response = requests.post(url, headers=headers, json=payload).json()
# print(response["choices"][0]["message"]["content"])

firm_desc = response["choices"][0]["message"]["content"].split('</think>')[-1]
firm_desc_reasoning = recr_email_reasoning = response['choices'][0]['message']['content'].split('<think>')[1].split('</think>')[0]
print(firm_desc)


<think>
Okay, I need to extract information about Ware Malcomb based on the provided search results. Let me start by going through each source to gather relevant details.

First, from source [1], Ware Malcomb is described as a dynamic commercial real estate design firm founded in 1972. They offer architecture, planning, interior design, and civil engineering services. Their approach is multidisciplinary and focuses on innovative solutions. Leadership includes CEO Ken Wink and President Jay Todisco.

Source [2] mentions their expansion into multi-family housing and use of TestFit software to save time on site planning. They saved $200k-$300k in non-billable hours by automating site feasibility studies. This shows their adoption of technology for efficiency.

Source [3] provides financials: $103.8M revenue, 983 employees, and locations across the Americas. They're recognized in industry rankings like Engineering News-Record’s Top 500. Their specialties include commercial office, industri

In [ ]:
firm_desc

"\n\nWare Malcomb is a global commercial real estate design firm established in 1972, offering integrated architecture, interior design, civil engineering, branding, and building measurement services[1][3]. Headquartered in San Diego with over 28 offices across the Americas[2][3], the firm employs 983 professionals[3] and generates $103.8M in annual revenue[3].\n\n**Focus Areas & Specializations**  \n- **Market Sectors**: Industrial (including cold storage), office/corporate, multifamily housing, healthcare, retail, science & technology[1][6]  \n- **Core Services**:  \n  - Site feasibility studies using AI-powered tools like TestFit for 65% faster planning[2]  \n  - Sustainable design solutions[6]  \n  - Workplace strategy and space optimization[6]  \n  - Brand integration through environmental graphics[6]  \n\n**Design Philosophy**  \n- Combines innovation with practical solutions emphasizing long-term real estate value[1][6]  \n- Multidisciplinary approach integrating architecture wi

In [ ]:
class ArchFirm(BaseModel):
    recruiting_email: Optional[str] = Field(None, description="What is the recruiting or careers contact email address listed on the firm's website or careers page?")
    # firm_description: Optional[str] = Field(None, description="What is the detailed description of the architecture firm, including their focus areas, specializations, notable projects, design philosophy, and overall approach to architecture?")

payload = {
    "model": "sonar-reasoning-pro",
    "messages": [
        {"role": "system", "content": "For a given architecture firm, your task is to extract their recruiting email address as listed on their careers/contact page. Be precise. You must ONLY output a JSON object with the result."},
        {"role": "user", "content": (
            f"Recruiting email for : {firm_name}"
            "Please output a JSON object containing the following field: "
            "recruiting_email"
        )},
    ],
    "response_format": {
		    "type": "json_schema",
        "json_schema": {"schema": ArchFirm.model_json_schema()},
    },
}


response = requests.post(url, headers=headers, json=payload).json()
# print(response["choices"][0]["message"]["content"])

In [ ]:
json_str = response['choices'][0]['message']['content'].split('```json')[-1].split('```')[0].strip()
recruiting_email = json.loads(json_str)
recr_email_reasoning = response['choices'][0]['message']['content'].split('<think>')[1].split('</think>')[0]


{'recruiting_email': 'jkiddle@waremalcomb.com'}


#### (interm) generate pointers from resume + portfolio + firm description

In [ ]:
pointers_agent = Agent(sys_prompt= PROMPTS['generate_pointers'], model = "gemini-2.0-flash")

prompt = f"""
For the given architecture firm, and my resume and portfolio details, generate a detailed bullet-point list of all the key alignment areas that should be mentioned in a personalized email to the recruiter.
Firm Details: 
{firm_desc}

Resume: 
{resume}

Portfolio details:
{portfolio}
"""

email_pointers = pointers_agent.llm_call(prompt)

#### Write email

In [ ]:
PROMPTS.keys()

dict_keys(['generate_pointers', 'write_email', 'write_cover_letter', 'review_email'])

In [ ]:
email_agent = Agent(sys_prompt= PROMPTS['write_email)'], model = "gemini-2.0-flash")

prompt = f"""
Using the pointers provided and my resume as context, write a concise, personalized email to a recruiter at an architectural firm for an entry-level position. 


Pointers
{email_pointers}

Resume: 
{resume}
"""

email_draft = email_agent.llm_call(prompt)

#### Write Cover letter

In [ ]:
cl_agent = Agent(sys_prompt= PROMPTS['write_cover_letter'], model = "gemini-2.0-flash")

job_description = ""

prompt = f"""
Using the job description, personalized pointers provided, and my resume as context, write the perfect cover letter for a candidate applying to an architectural firm. If no job description is provided, just write a generic cover letter showing my education, professional and technical background and alignment with the firm for opportunities. 

Job Description:
{job_description}

Pointers
{email_pointers}

Resume: 
{resume}


"""

cl_draft = cl_agent.llm_call(prompt)

KeyError: 'write_cover_letter)'

In [1]:
from gspread_utils import *

In [4]:
x = GSpreadUtils(credentials_file='/Users/tarunnv/Documents/bp-ai-gsheet-80764f1a817f.json')
x.open_by_key('1pRhTtAB3AM1qHQMhdJvBncyAAzYvGeD2N94qScftCZE')

records = x.get_all_records('arch_firms')

In [15]:
x.sh.worksheet('arch_firms').append_row([1, 2, 3, 5])

{'spreadsheetId': '1pRhTtAB3AM1qHQMhdJvBncyAAzYvGeD2N94qScftCZE',
 'tableRange': 'arch_firms!A1:D3',
 'updates': {'spreadsheetId': '1pRhTtAB3AM1qHQMhdJvBncyAAzYvGeD2N94qScftCZE',
  'updatedRange': 'arch_firms!A4:D4',
  'updatedRows': 1,
  'updatedColumns': 4,
  'updatedCells': 4}}

In [7]:
x.sh.worksheets()

[<Worksheet 'arch_firms' id:0>]